In [15]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import json
import requests
import os
#import Levenshtein as lev
mu.css()

In [16]:
AVSNITTMAPPE = "Avsnitt_november/"

In [17]:
#!pip install fuzzywuzzy

In [18]:
#!pip install python-Levenshtein

In [19]:
from fuzzywuzzy import fuzz

In [20]:
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

In [21]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs


In [22]:
def find_theme(title, themes):
    from collections import Counter
    dist = Counter()
    for th in themes:
        dist[th] = fuzz.ratio(th, title)
    x = dist.most_common(1)[0][0]
    if dist[x] > 60:
        res = x
    else:
        res = ""
    return res

In [23]:
def show_paras(title, themes):
    avsnitt = json.load(open(title))
    paras = get_para(avsnitt)
    theme = find_theme(title, themes)
    words = [w for hyperlexeme in themes[theme] for w in themes[theme][hyperlexeme]]
    ptext = "<h2>" + theme + "</h2>"
    ptext += "<h3" + ', '.join(themes[theme].keys()) + "</h3>"
    for urn in paras:
        ptext += "<h4> <a href='" + link(urn) +"'" + "target=_>" + ', '.join([str(x) for x in nb.metadata(urn)[0][1:4]]) + "</a>"
        for paranum in paras[urn]:
            ptext += "<h4>" + paranum + '</h4>'
            modified = paras[urn][paranum]
            #print(modified)
            for w in words:
                #print(w)
                modified = modified.replace(" "+w+" ", " <b>" + w + "</b> ")
            ptext += modified + '\n\n'
    return ptext

In [24]:
with open('heidis_temadefinisjoner_november.json') as f:
    themes = json.load(f)

In [25]:
link = lambda x: "https://urn.nb.no/URN:NBN:no-nb_digibok_" + str(x)

In [26]:
print(len(themes))
for th in themes:
    print(th)

5
“Kvindens Sandhed” (Minst 3 av 4).
“Kvinde, lesning og religion” (Minst 4 av 5)
“Forfatterinder og religion”  (Minst 3 av 4)
“Kvindens posisjon” (Minst 4 av 6)
“Kvinden og offentlig hygiene” (Minst 4 av 6)


In [27]:
#themes

In [28]:
#themes[find_theme('forfatter og religion', themes)]

In [29]:
import os
r,d,f = next(os.walk("."))
jsons = [fs for fs in f if fs.endswith('.json')]
files = {f:find_theme(f, themes) for f in jsons if find_theme(f, themes) != ''}

In [30]:
print(len(files))
files

5


{'Forfatterinder og religion  Minst 3 av 4.json': '“Forfatterinder og religion”  (Minst 3 av 4)',
 'Kvinde lesning og religion Minst 4 av 5.json': '“Kvinde, lesning og religion” (Minst 4 av 5)',
 'Kvinden og offentlig hygiene Minst 4 av 6.json': '“Kvinden og offentlig hygiene” (Minst 4 av 6)',
 'Kvindens posisjon Minst 4 av 6.json': '“Kvindens posisjon” (Minst 4 av 6)',
 'Kvindens Sandhed Minst 3 av 4.json': '“Kvindens Sandhed” (Minst 3 av 4).'}

In [31]:
for f in files:
    print(f.split('.json')[0])

Forfatterinder og religion  Minst 3 av 4
Kvinde lesning og religion Minst 4 av 5
Kvinden og offentlig hygiene Minst 4 av 6
Kvindens posisjon Minst 4 av 6
Kvindens Sandhed Minst 3 av 4


In [32]:
res = ""
for f in files:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = show_paras(f, themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Forfatterinder og religion  Minst 3 av 4.json
Kvinde lesning og religion Minst 4 av 5.json
Kvinden og offentlig hygiene Minst 4 av 6.json
Kvindens posisjon Minst 4 av 6.json
Kvindens Sandhed Minst 3 av 4.json
